1개 폴더에 1000개의 mel 데이터가 존재하여

최소한의 데이터 불러오기를 위하여

폴더 단위의 멜 데이터를 하나의 멜 데이터 파일로 만듬

(코랩의 경우 데이터 불러오기시 많은 시간이 발생함)

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings(action='ignore')

data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Data/'
music_data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Music_Data/'
mel_data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Mel_Data/'

In [ ]:
song_meta_df = pd.read_json(data_dir + 'song_meta.json', typ = 'frame')
song_meta_df = song_meta_df.sort_values('id')

In [ ]:
import math
import pickle
import gc

'''
우리가 만들 피일의 경우 접근방식
floor(song_id / 1000).npy <- 파일명
각 인덱스 : song_id % 1000 <- 파일 내 곡 위치
'''

for file_name in tqdm(range(220, 221)):
    get_song_id_li = song_id_li[file_name * 1000: (file_name + 1) * 1000]
    get_mel_li = []
    for get_song_id in tqdm(get_song_id_li):
        # 데이터 불러오기
        try: 
            mel = np.load(music_data_dir + f'arena_mel/{math.floor(get_song_id / 1000)}/{get_song_id}.npy')
        except ValueError as m:
            mel = np.zeros((48, 1876), dtype = np.float32)
            print(ValueError, get_song_id)

        # 크기 확인
        axis_0, axis_1 = mel.shape
        if axis_0 * axis_1 != 48 * 1876:
            r = 1876 - axis_1
            mel = np.concatenate([mel, np.tile(mel[:, -1], (r, 1)).T], axis = 1)
        
        get_mel_li.append(mel.reshape(1, 48, 1876))

    # 저장
    get_mel_li = np.concatenate(get_mel_li)
    with open(mel_data_dir + f'{file_name}.pickle', 'wb') as f:
        pickle.dump(get_mel_li, f)
    
    # 데이터 확인
    print(f'file_name : {file_name}, size:', end = '')
    print(get_mel_li.shape)

    # 메모리 정리
    del get_song_id_li
    del get_mel_li
    del mel

    gc.collect()